In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '../bar_movement/data/'

In [3]:
currency_pair = 'Eur_Usd'

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date, utc=True)
df.reset_index(drop=True, inplace=True)

In [5]:
def atr(highs, lows, closes, lookback=14):
    high_low = highs - lows
    high_close = np.abs(highs - closes.shift())
    low_close = np.abs(lows - closes.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback

In [6]:
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
currency1, currency2 = currency_pair.split('_')
currency1, currency2 = currency1.upper(), currency2.upper()
news = pd.read_csv(file_path + 'events.csv')
news.Date = pd.to_datetime(news.Date, utc=True)
news = news.loc[(news['Currency_Code'] == currency1) | (news['Currency_Code'] == currency2)]
news.drop(news[(news['Impact'] != 'low') & (news['Impact'] != 'med') & (news['Impact'] != 'high')].index, inplace=True)
news.loc[news['Impact'] == 'low', 'Impact'] = 1
news.loc[news['Impact'] == 'med', 'Impact'] = 2
news.loc[news['Impact'] == 'high', 'Impact'] = 3
news['Impact'] = pd.to_numeric(news['Impact'])
news.drop(['Currency_Code', 'Actual', 'Previous', 'Actual_Val', 'Forecast_Val', 'Previous_Val'], axis=1, inplace=True)
by_date = news.groupby('Date')
impact = by_date['Impact'].max().reset_index()
news = news.iloc[0:0]
news['Date'], news['Impact'] = impact['Date'], impact['Impact']
news.tail()

/var/folders/y1/r6lw81g960n148db75yhq7140000gn/T/ipykernel_8299/521418843.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news['Date'], news['Impact'] = impact['Date'], impact['Impact']


,Date,Impact
7828,2022-09-09 00:00:00+00:00,2
7829,2022-09-09 06:45:00+00:00,2
7830,2022-09-09 14:00:00+00:00,2
7831,2022-09-09 16:00:00+00:00,2
7832,2022-09-09 17:00:00+00:00,1


In [8]:
df = pd.merge(df, news, how='left', on='Date')
df.head()

,Date,Bid_Open,Bid_High,Bid_Low,Bid_Close,Ask_Open,Ask_High,Ask_Low,Ask_Close,Mid_Open,Mid_High,Mid_Low,Mid_Close,Volume,atr,Impact
0,2021-04-01 07:05:00+00:00,1.17246,1.17252,1.17206,1.17238,1.17258,1.17265,1.17218,1.17251,1.17252,1.17258,1.17212,1.17244,524,0.000484,NaN
1,2021-04-01 07:10:00+00:00,1.17239,1.17334,1.17239,1.17316,1.17252,1.17348,1.17252,1.17330,1.17246,1.17341,1.17246,1.17323,662,0.000531,NaN
2,2021-04-01 07:15:00+00:00,1.17315,1.17321,1.17274,1.17297,1.17329,1.17334,1.17287,1.17310,1.17322,1.17328,1.17281,1.17304,490,0.000520,NaN
3,2021-04-01 07:20:00+00:00,1.17298,1.17362,1.17263,1.17362,1.17311,1.17375,1.17276,1.17375,1.17304,1.17368,1.17270,1.17368,548,0.000574,NaN
4,2021-04-01 07:25:00+00:00,1.17364,1.17420,1.17352,1.17414,1.17377,1.17433,1.17366,1.17427,1.17370,1.17426,1.17359,1.17420,698,0.000598,NaN


In [9]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]
rounding = 5 if 'Jpy' not in currency_pair else 3
divider = 10000 if 'Jpy' not in currency_pair else 100

In [10]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(open_price, stop_loss, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = abs(open_price - stop_loss)
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, atr_multiplier, n_minutes, impact_cutoff):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    lo = None
    trade = None
    trade_lengths = []

    for i in range(1, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_mid_open, curr_mid_low = \
            df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_Low']]
        ask_low, bid_high, atr = df.loc[df.index[i - 1], ['Ask_Low', 'Bid_High', 'atr']]
        impact = df.loc[df.index[i], 'Impact']
        important_news = not np.isnan(impact) and impact >= impact_cutoff

        if lo is not None:
            minutes_elapsed = (curr_date - lo['start_date']).total_seconds() / 60
            stop_signal = minutes_elapsed >= n_minutes

            if stop_signal:
                lo = None

        if lo is None and trade is None and important_news:
            range_high, range_low = bid_high + (atr * atr_multiplier), ask_low - (atr * atr_multiplier)
            rang = abs(float(range_high) - float(range_low))
            pips_gain = (rang / 2) * risk_reward_ratio
            pips_to_risk = pips_gain / risk_reward_ratio
            
            if range_high > range_low and spread <= pips_to_risk * spread_cutoff:
                buy_open_price = float(range_low)
                buy_stop_gain = curr_ao
                buy_pips_gained = buy_stop_gain - buy_open_price
                buy_pips_to_risk = buy_pips_gained / risk_reward_ratio
                buy_pullback = buy_open_price - buy_pips_to_risk
                buy_stop_loss = round(buy_pullback, 5)
                buy_mo = buy_open_price - (spread / 2)
                buy_n_units = get_n_units(buy_open_price, buy_stop_loss, buy_mo, currency_pair)

                sell_open_price = float(range_high)
                sell_stop_gain = curr_bo
                sell_pips_gained = sell_open_price - sell_stop_gain
                sell_pips_to_risk = sell_pips_gained / risk_reward_ratio
                sell_pullback = sell_open_price + sell_pips_to_risk
                sell_stop_loss = round(sell_pullback, 5)
                sell_mo = sell_open_price + (spread / 2)
                sell_n_units = get_n_units(sell_open_price, sell_stop_loss, sell_mo, currency_pair)

                lo = {'buy_open_price': buy_open_price, 'buy_stop_gain': buy_stop_gain, 'buy_stop_loss': buy_stop_loss, 'buy_n_units': buy_n_units, 
                               'sell_open_price': sell_open_price, 'sell_stop_gain': sell_stop_gain, 'sell_stop_loss': sell_stop_loss, 'sell_n_units': sell_n_units, 'start_date': curr_date}

        if trade is None and lo is not None:
            place_buy = curr_ask_low <= lo['buy_open_price']
            place_sell = curr_bid_high >= lo['sell_open_price']

            if place_buy:
                open_price = lo['buy_open_price']
                stop_gain = lo['buy_stop_gain']
                pips_gained = stop_gain - open_price
                curr_pips_to_risk = pips_gained / risk_reward_ratio
                stop_loss = lo['buy_stop_loss']

                if stop_loss < open_price and stop_gain > open_price:
                    n_units = lo['buy_n_units']

                    trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'index': i}

                    n_buys += 1

                    pips_risked.append(curr_pips_to_risk)
                    
                    lo = None

            elif place_sell:
                open_price = lo['sell_open_price']
                stop_gain = lo['sell_stop_gain']
                pips_gained = open_price - stop_gain
                curr_pips_to_risk = pips_gained / risk_reward_ratio
                stop_loss = lo['sell_stop_loss']

                if stop_loss > open_price and stop_gain < open_price:
                    n_units = lo['sell_n_units']

                    trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                            'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'index': i}

                    n_sells += 1

                    pips_risked.append(curr_pips_to_risk)

                    lo = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1
            trade_lengths.append(i - trade['index'] + 1)

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1
            trade_lengths.append(i - trade['index'] + 1)

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1
            trade_lengths.append(i - trade['index'] + 1)

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1
            trade_lengths.append(i - trade['index'] + 1)

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, trade_lengths

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.0, 1.5, 2.0]
spread_cutoffs = [0.10, 0.20]
atr_multipliers = [1.0, 1.5, 2.0, 2.5, 3.0]
n_minutes_vals = [5, 10, 15, 30]
impact_cutoffs = [2, 3]

all_combos = []


for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for atr_multiplier in atr_multipliers:
            for n_minutes in n_minutes_vals:
                for impact_cutoff in impact_cutoffs:
                    all_combos.append((risk_reward_ratio, spread_val, atr_multiplier, n_minutes, impact_cutoff))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_atr_multiplier = None
best_n_minutes= None
best_impact_cutoff = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, atr_multiplier, n_minutes, impact_cutoff in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, trade_lengths = run_simulation(risk_reward_ratio, spread_val, atr_multiplier, n_minutes, impact_cutoff)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    avg_pips_risked = (sum(pips_risked) / len(pips_risked)) if len(pips_risked) > 0 else 'N/A'
    avg_trade_len = (sum(trade_lengths) / len(trade_lengths)) if len(trade_lengths) > 0 else 0
    print('Avg pips risked: ' + str(avg_pips_risked))
    print('Avg trade length: ' + str(avg_trade_len))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread_val': spread_val, 'atr_multiplier': atr_multiplier, 'n_minutes': n_minutes, 'impact_cutoff': impact_cutoff})


    if reward > best_reward and avg_trade_len >= 1:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_atr_multiplier = atr_multiplier
        best_n_minutes = n_minutes
        best_impact_cutoff = impact_cutoff

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 240

-316.21693500002124
Num buys: 18
Num sells: 24
Num trades: 42
Num wins: 12
Num losses: 30
Win streak: 2
Loss streak: 6
Avg pips risked: 0.0008861309523809606
Avg trade length: 22.38095238095238
Remaining runs: 239
Best reward so far: -316.21693500002124

-398.15211928555885
Num buys: 16
Num sells: 10
Num trades: 26
Num wins: 6
Num losses: 20
Win streak: 1
Loss streak: 6
Avg pips risked: 0.0005305082417582694
Avg trade length: 2.3846153846153846
Remaining runs: 238
Best reward so far: -316.21693500002124

-130.46378999998214
Num buys: 29
Num sells: 23
Num trades: 52
Num wins: 25
Num losses: 27
Win streak: 4
Loss streak: 4
Avg pips risked: 0.0021293131868131976
Avg trade length: 58.23076923076923
Remaining runs: 237
Best reward so far: -130.46378999998214

-512.9992950000067
Num buys: 24
Num sells: 24
Num trades: 48
Num wins: 19
Num losses: 29
Win streak: 4
Loss streak: 7
Avg pips risked: 0.0018131994047618967
Avg trade length: 40.791666666666664
Remaining runs: 236
Best r

In [15]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best impact cutoff: ' + str(best_impact_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 590.5448557143058
Best risk/reward ratio: 1.5
Best spread: 0.2
Best atr multiplier: 2.0
Best n minutes: 10
Best adx cutoff: 2
-----------------------
Top results:
{'reward': 314, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_minutes': 5, 'impact_cutoff': 2}
{'reward': 345, 'ratio': 1.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_minutes': 5, 'impact_cutoff': 2}
{'reward': 195, 'ratio': 2.0, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_minutes': 15, 'impact_cutoff': 3}
{'reward': 319, 'ratio': 1.5, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_minutes': 10, 'impact_cutoff': 3}
{'reward': 313, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_minutes': 10, 'impact_cutoff': 3}
{'reward': 251, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 1.5, 'n_minutes': 15, 'impact_cutoff': 3}
{'reward': 216, 'ratio': 1.5, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_minutes': 5, 'impact_cutoff': 3}
{'reward': 436, 'ratio':

In [12]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best atr multiplier: ' + str(best_atr_multiplier))
print('Best n minutes: ' + str(best_n_minutes))
print('Best impact cutoff: ' + str(best_impact_cutoff))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 632.8377100001028
Best risk/reward ratio: 2.0
Best spread: 0.2
Best atr multiplier: 2.5
Best n minutes: 30
Best impact cutoff: 3
-----------------------
Top results:
{'reward': 247, 'ratio': 1.5, 'spread_val': 0.2, 'atr_multiplier': 2.5, 'n_minutes': 5, 'impact_cutoff': 3}
{'reward': 149, 'ratio': 2.0, 'spread_val': 0.1, 'atr_multiplier': 2.5, 'n_minutes': 5, 'impact_cutoff': 2}
{'reward': 393, 'ratio': 2.0, 'spread_val': 0.2, 'atr_multiplier': 2.5, 'n_minutes': 5, 'impact_cutoff': 3}
{'reward': 192, 'ratio': 2.0, 'spread_val': 0.2, 'atr_multiplier': 2.5, 'n_minutes': 10, 'impact_cutoff': 3}
{'reward': 200, 'ratio': 2.0, 'spread_val': 0.1, 'atr_multiplier': 2.0, 'n_minutes': 5, 'impact_cutoff': 2}
{'reward': 196, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 2.0, 'n_minutes': 5, 'impact_cutoff': 2}
{'reward': 198, 'ratio': 1.0, 'spread_val': 0.2, 'atr_multiplier': 2.5, 'n_minutes': 5, 'impact_cutoff': 3}
{'reward': 419, 'ratio':